In [1]:
import sys

import clip
import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm

sys.path.append('/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML')
from data_preprocess import data_selection

In [2]:
gender_code = { 0 : 'male', 1 : 'female'}
race_code = { 0 : 'white', 1 : 'black', 2 : 'asian', 3 : 'indian', 4 : 'other'}

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
device

'cpu'

`device` will indicate wich `CLIP` model to use depending on the available hardware. If there is a GPU, `cuda:0` will be used  or `cuda:1` if there are multiple GPUs. If there is not a GPU, `cpu` will be used.

In [4]:
race_labels = ['black', 'white', 'asian', 'indian', 'other']
race_tkns = ['A photo of a person of color ' + label for label in race_labels]
race_text = clip.tokenize(race_tkns).to(device)

In [5]:
sex_labels = ['male', 'female']
sex_tkns = ['A photo of a person of sex ' + label for label in sex_labels]
sex_text = clip.tokenize(sex_tkns).to(device)

`tkns` is the domain of possible values. `CLIP` model predicts for each image the most probable sentence from `tkns`, in this case. 

Initializing usefull variables for `CLIP` model application.

In [7]:
BATCH_SIZE = 64

dir_path = r'/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/'
photo_paths = data_selection(dir_path)[1]['filepath']
photo_paths

/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data_preprocess.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][i]= GENDER_MAPPER[df['gender'][i]]
/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data_preprocess.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'][i]= RACE_MAPPER[df['race'][i]]


2        /Users/hanselblanco/Documents/4to/ML/project/b...
9        /Users/hanselblanco/Documents/4to/ML/project/b...
10       /Users/hanselblanco/Documents/4to/ML/project/b...
12       /Users/hanselblanco/Documents/4to/ML/project/b...
13       /Users/hanselblanco/Documents/4to/ML/project/b...
                               ...                        
23672    /Users/hanselblanco/Documents/4to/ML/project/b...
23673    /Users/hanselblanco/Documents/4to/ML/project/b...
23681    /Users/hanselblanco/Documents/4to/ML/project/b...
23700    /Users/hanselblanco/Documents/4to/ML/project/b...
23705    /Users/hanselblanco/Documents/4to/ML/project/b...
Name: filepath, Length: 4742, dtype: object

In [8]:
filenames = []
for path in photo_paths:
    path = path.split('/')
    filename = path[-1]
    filenames.append(filename)
    splitted = filename.split('_')
    age = splitted[0]
    gender = splitted[1]
    race = splitted[2]

In [9]:
df = pd.DataFrame(filenames, columns = ['filename'] )
df['filepath'] = df.filename.apply(lambda x: dir_path + x)
df['gender'] = df.filename.apply(lambda x: gender_code[int(x.split('_')[1])])
df['race'] = df.filename.apply(lambda x: race_code[int(x.split('_')[-2])])
df.head()

,filename,filepath,gender,race
0,86_1_0_20170120225751953.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,female,white
1,36_0_3_20170119180245724.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,male,indian
2,58_0_2_20170116193704928.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,male,asian
3,35_1_2_20170116185947151.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,female,asian
4,1_1_3_20161219230734016.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,female,indian


Executing `CLIP` model for `photos_to_analize` images from the dataset.

In [12]:
sex_results = []
race_results = []

photos = []
for photo_path in photo_paths:
    photos.append(Image.open(photo_path))

pending_photos = len(photos)
for i in tqdm(range(0, len(photos), min(BATCH_SIZE, pending_photos))):
    pending_photos = len(photos) - i
    images = [preprocess(photos[photo_idx]) for photo_idx in range(i, min(i + BATCH_SIZE, len(photos)))]
    image_input = torch.tensor(np.stack(images)).to(device)
    with torch.no_grad():
        sex_logits_per_image, sex_logits_per_text = model(image_input, sex_text)
        race_logits_per_image, race_logits_per_text = model(image_input, race_text)
        
        # The softmax function takes the original confidence and applys a transform to make all the confidence add up to one
        sex_probs = sex_logits_per_image.softmax(dim=-1).cpu().numpy()
        race_probs = race_logits_per_image.softmax(dim=-1).cpu().numpy()
        
        sex_results.append(sex_probs)
        race_results.append(race_probs)

100%|██████████| 75/75 [03:05<00:00,  2.47s/it]


In [13]:
race_res = np.concatenate(race_results, axis=0)
race_choices = np.argmax(race_res, axis=1)

sex_res = np.concatenate(sex_results, axis=0)
sex_choices = np.argmax(sex_res, axis=1)

In [14]:
r_getlabel = lambda x:race_labels[x]
r_vgetlabel = np.vectorize(r_getlabel)
races = r_vgetlabel(race_choices)

s_getlabel = lambda x:sex_labels[x]
s_vgetlabel = np.vectorize(s_getlabel)
genders = s_vgetlabel(sex_choices)
len(genders)

4742

`races` is the vector with predicted labels to add to each sentence from `race_tkns` for each image, ordered.

`genders` is the vector with predicted labels to add to each sentence from `sex_tkns` for each image, ordered.

In [15]:
df['predicted_gender'] = genders
df['predicted_race'] = races
df.head()

,filename,filepath,gender,race,predicted_gender,predicted_race
0,86_1_0_20170120225751953.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,female,white,female,white
1,36_0_3_20170119180245724.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,male,indian,female,indian
2,58_0_2_20170116193704928.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,male,asian,male,asian
3,35_1_2_20170116185947151.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,female,asian,female,asian
4,1_1_3_20161219230734016.jpg.chip.jpg,/Users/hanselblanco/Documents/4to/ML/project/b...,female,indian,male,asian


# Detecting bias in `CLIP` results.

## Race (Disparate impact)

### Selection Rate (Positive results / N)

In [16]:
totals = {'white': 0, 'black': 0, 'asian': 0, 'indian': 0, 'other': 0}
positives = {'white': 0, 'black': 0, 'asian': 0, 'indian': 0, 'other': 0}

for i in range(len(photos)):
    true_race = df['race'][i]
    predicted_race = df['predicted_race'][i]
    totals[true_race] += 1
    if true_race == predicted_race:
        positives[predicted_race] += 1
                
whites_sr, blacks_sr, asians_sr, indians_sr, others_sr = positives['white']/ totals['white'], positives['black']/ totals['black'], positives['asian']/ totals['asian'], positives['indian']/ totals['indian'], positives['other']/ totals['other']

whites_sr, blacks_sr, asians_sr, indians_sr, others_sr
    

(0.8724565756823821,
 0.7737306843267108,
 0.9519650655021834,
 0.7723270440251573,
 0.08849557522123894)

#### Disparate impact

In [17]:

# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact_b_w = blacks_sr/ whites_sr
disp_impact_b_a = asians_sr/ whites_sr
disp_impact_b_i = indians_sr/ whites_sr
disp_impact_b_w, disp_impact_b_a, disp_impact_b_i

(0.8868414840263494, 1.0911317445886801, 0.8852326471619408)

In [18]:
if disp_impact_b_w < 0.8:
    print('Disparate impact present in black group / white group')
if disp_impact_b_a < 0.8:
    print('Disparate impact present in asian group / white group')
if disp_impact_b_i < 0.8:
    print('Disparate impact present in indian group / white group')

## Sex (Disparate impact)

### Selection Rate (Positive results / N)

In [19]:
totals = {'male': 0, 'female': 0}
positives = {'male': 0, 'female': 0}

for i in range(len(photos)):
    true_gender = df['gender'][i]
    predicted_gender = df['predicted_gender'][i]
    totals[true_gender] += 1
    if true_gender == predicted_gender:
        positives[predicted_gender] += 1
                
males_sr, females_sr = positives['male']/ totals['male'], positives['female']/ totals['female']

males_sr, females_sr

(0.9536103267446551, 0.9628811312417145)

#### Disparate impact

In [20]:
# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact = females_sr / males_sr
disp_impact

1.0097217954095645

In [21]:
if disp_impact < 0.8:
    print('Disparate impact present in female group / male group')


## Sex (Equalized odds)

In [22]:
tp_males, tp_females, fn_males, fn_females, fp_males, fp_females = 0, 0, 0, 0, 0, 0

for i in range(len(photos)):
    data = df['filename'][i].split('_')
    gender_number = int(data[1])
    match gender_code[gender_number]:
        case 'male':
            if genders[i] == 'male':
                tp_males += 1
            else:
                fp_females += 1
                fn_males += 1 # False negative (wrong no male prediction, in this case, equal to female false positive)
        case 'female':
            if genders[i] == 'female':
                tp_females += 1
            else:
                fp_males += 1
                fn_females += 1
                
males_tpr, females_tpr = tp_males/ (tp_males + fn_males), tp_females/ (tp_females + fn_females)

males_fpr, females_fpr = fp_males/ (fp_males + fn_males), fp_females/ (fp_females + fn_females)


#### True Positive Rates

In [23]:
males_tpr, females_tpr

(0.9536103267446551, 0.9628811312417145)

In [24]:
if abs(males_tpr - females_tpr) < 0.15:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_tpr - females_tpr))

Equalized odds


#### False Positive Rates

In [25]:
males_fpr, females_fpr

(0.4221105527638191, 0.5778894472361809)

In [26]:
if abs(males_fpr - females_fpr) < 0.15:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_fpr - females_fpr))

Not equalized odds
0.15577889447236176


## Race (Equalized odds)

In [27]:
race_rates = {'white': {'tp': 0, 'fp': 0, 'fn': 0}, 'black': {'tp': 0, 'fp': 0, 'fn': 0}, 'asian': {'tp': 0, 'fp': 0, 'fn': 0}, 'indian': {'tp': 0, 'fp': 0, 'fn': 0}, 'other': {'tp': 0, 'fp': 0, 'fn': 0}}

for i in range(len(photos)):
    race = df['race'][i]
    pred_race = df['predicted_race'][i]
    
    if race == pred_race:
        race_rates[race]['tp'] += 1
    else:
        race_rates[race]['fn'] += 1
        race_rates[pred_race]['fp'] += 1

tpr = lambda tp, fn: tp/ (tp + fn)
fpr = lambda fp, fn: fp/ (fp + fn)

tpr_values = {'white': 0, 'black': 0, 'asian': 0, 'indian': 0, 'other': 0}
fpr_values = {'white': 0, 'black': 0, 'asian': 0, 'indian': 0, 'other': 0}
for race in race_rates.keys():
    rates = race_rates[race]
    tpr_values[race] = tpr(rates['tp'], rates['fn'])
    fpr_values[race] = fpr(rates['fp'], rates['fn'])

True Positive Rates

In [28]:
[tpr_value for tpr_value in tpr_values.values()]

[0.8724565756823821,
 0.7737306843267108,
 0.9519650655021834,
 0.7723270440251573,
 0.08849557522123894]

In [29]:
from itertools import combinations

equalized_odds = True

for pair in combinations(tpr_values.keys(), 2):
    first_race = pair[0]
    second_race = pair[1]
    if first_race == 'other' or second_race == 'other':
        continue
    if abs(tpr_values[first_race] - tpr_values[second_race]) >= 0.1:
        equalized_odds = False
        print('Not equalized odds between ' + first_race + ' and ' + second_race)

if equalized_odds:
    print('Equalized odds')

Not equalized odds between white and indian
Not equalized odds between black and asian
Not equalized odds between asian and indian


False Positive Rates

In [30]:
[fpr_value for fpr_value in fpr_values.values()]

[0.6015503875968993,
 0.2906574394463668,
 0.8186813186813187,
 0.44135802469135804,
 0.4169811320754717]

In [31]:
from itertools import combinations

equalized_odds = True

for pair in combinations(fpr_values.keys(), 2):
    first_race = pair[0]
    second_race = pair[1]
    if first_race == 'other' or second_race == 'other':
        continue
    if abs(fpr_values[first_race] - fpr_values[second_race]) >= 0.1:
        equalized_odds = False
        print('Not equalized odds between ' + first_race + ' and ' + second_race)

if equalized_odds:
    print('Equalized odds')

Not equalized odds between white and black
Not equalized odds between white and asian
Not equalized odds between white and indian
Not equalized odds between black and asian
Not equalized odds between black and indian
Not equalized odds between asian and indian
